Here we'll show a bit about how the convex hull algorithm works for 3D point sets.
As we'll see shortly, computing convex hulls is much more difficult in higher dimensions than in 2D because visibility gets more interesting.

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=1729)
num_points = 40
X = rng.normal(size=(num_points, 3))

In [ ]:
import pyvista
pyvista.set_plot_theme("document")
plotter = pyvista.Plotter(notebook=True)
plotter.add_points(X, render_points_as_spheres=True, point_size=10)
plotter.show()

In [ ]:
import zmsh
hull_machine = zmsh.ConvexHullMachine(X)

In [ ]:
from copy import deepcopy
geometries = [deepcopy(hull_machine.geometry)]
visible_cells_ids = []

while not hull_machine.is_done():
    vertex_id, cell_ids = hull_machine.visible.get_next_vertex_and_cells()
    visible_cells_ids.append(cell_ids)
    hull_machine.step()
    geometries.append(deepcopy(hull_machine.geometry))
    
visible_cells_ids.append([])

We're doing all our visualization here with [pyvista](https://docs.pyvista.org/), which provides a nice frontend to the Visualization Toolkit or VTK.
This package expects the faces of a 3D mesh to be passed in a certain way; the helper function below does the right thing for VTK.

In [ ]:
def topology_to_vtk(topology):
    faces = []
    cells = topology.cells(topology.dimension)
    for cell_id in range(len(cells)):
        try:
            face_ids, matrices = cells.closure(cell_id)
            vertices = face_ids[0]
            faces.append([len(vertices)] + list(vertices))
        except IndexError:
            faces.append([0])

    return np.hstack(faces)

Now we can visualize the progress of the algorithm.

In [ ]:
plotter = pyvista.Plotter(notebook=True)
plotter.add_points(X, render_points_as_spheres=True, point_size=10)

faces = [topology_to_vtk(geometry.topology) for geometry in geometries]
kwargs = {
    "name": "hull",
    "show_edges": True,
    "line_width": 5,
    "show_scalar_bar": False,
}
def create_mesh(index):
    surface = pyvista.PolyData(X, faces[int(index)])
    num_cells = len(geometries[int(index)].topology.cells(2))
    colors = np.zeros(num_cells)
    colors[visible_cells_ids[int(index)]] = 1.0
    plotter.add_mesh(surface, scalars=colors, **kwargs)
    return

plotter.add_slider_widget(create_mesh, [0, len(faces)], value=0, title="step")
plotter.show()

Computing convex hulls in dimension 3 or higher is much more difficult in the 2D case because now multiple hull faces can be visible to the same extreme point.